In [3]:
dataset1 = []

class DataPoint:
  def __init__(self, status, vars):
    self.isTrue = status
    self.title = vars[0]
    self.text = vars[1]
    self.subject = vars[2]
    self.data = vars[3]

  def print(self):
    print(self.isTrue + " - " + self.title + " - " + self.text)
    



In [4]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(dataset1)


      label                                               text
0      fake  Donald Trump just couldn t wish all Americans ...
1      fake  House Intelligence Committee Chairman Devin Nu...
2      fake  On Friday, it was revealed that former Milwauk...
3      fake  On Christmas day, Donald Trump announced that ...
4      fake  Pope Francis used his annual Christmas Day mes...
5      fake  The number of cases of cops brutalizing and ki...
6      fake  Donald Trump spent a good portion of his day a...
7      fake  In the wake of yet another court decision that...
8      fake  Many people have raised the alarm regarding th...
9      fake  Just when you might have thought we d get a br...
10     fake  A centerpiece of Donald Trump s campaign, and ...
11     fake  Republicans are working overtime trying to sel...
12     fake  Republicans have had seven years to come up wi...
13     fake  The media has been talking all day about Trump...
14     fake  Abigail Disney is an heiress with brass ov

In [5]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_lg
!{sys.executable} -m spacy download en_core_web_sm

^C
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
LGVars = pd.DataFrame()
import numpy as np
import spacy as sp
import pandas as pd

# Load English tokenizer, tagger, parser and NER
nlp = sp.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []
for t in doc:
    row = [t.text, t.lemma_, t.pos_, sp.explain(t.pos_), t.is_stop]
    rows.append(row)
df = pd.DataFrame(rows, columns=cols)
print(df)

         text      lemma    POS                    explain  stopword
0        When       when  SCONJ  subordinating conjunction      True
1   Sebastian  Sebastian  PROPN                proper noun     False
2       Thrun      Thrun  PROPN                proper noun     False
3     started      start   VERB                       verb     False
4     working       work   VERB                       verb     False
5          on         on    ADP                 adposition      True
6        self       self   NOUN                       noun     False
7           -          -  PUNCT                punctuation     False
8     driving      drive   VERB                       verb     False
9        cars        car   NOUN                       noun     False
10         at         at    ADP                 adposition      True
11     Google     Google  PROPN                proper noun     False
12         in         in    ADP                 adposition      True
13       2007       2007    NUM   

In [7]:
import sys
!{sys.executable} -m pip install xgboost

In [11]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# Import libraries
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

print(dataset1.head(5))

nltk.download('stopwords')

def text_preprocess(message):
    # Remove punctuations
    nopunc = [char for char in message if char not in string.punctuation]

    # Join the characters again
    nopunc = "".join(nopunc)
    nopunc = nopunc.lower()

    # Remove any stopwords and non-alphabetic characters
    nostop = [
        word
        for word in nopunc.split()
        if word.lower() not in stopwords.words("english") and word.isalpha()
    ]

    return nostop


false_text = dataset1[dataset1["label"] == "fake"]["text"]
true_text = dataset1[dataset1["label"] == "true"]["text"]
print(f"Number of fake messages: {len(false_text)}")
print(f"Number of true messages: {len(true_text)}")



  label                                               text
0  fake  Donald Trump just couldn t wish all Americans ...
1  fake  House Intelligence Committee Chairman Devin Nu...
2  fake  On Friday, it was revealed that former Milwauk...
3  fake  On Christmas day, Donald Trump announced that ...
4  fake  Pope Francis used his annual Christmas Day mes...
Number of fake messages: 23452
Number of true messages: 21417


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jjfri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
####################################################################### TAKES FOREVER TO PROSESS
from sklearn.utils import shuffle
data_shuffled = dataset1.sample(len(dataset1))
#dataset1 = shuffle(dataset1)
#dataset1 = dataset1.iloc[0:500] # 0.0878 seconds per row // 89.922 min for entire dataset

# Remove punctuations/stopwords from all messages
data_shuffled["text"] = data_shuffled["text"].apply(text_preprocess)
data_shuffled.head(20)



In [ ]:
# Convert messages (as lists of string tokens) to strings
data_shuffled["text"] = data_shuffled["text"].agg(lambda x: " ".join(map(str, x)))
data_shuffled.head()

# Initialize count vectorizer
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(data_shuffled["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

# Convert strings to vectors using BoW
dataset1_bow = bow_transformer.transform(data_shuffled["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {dataset1_bow.shape}")
print(f"Amount of non-zero occurrences: {dataset1_bow.nnz}")



NameError: name 'data_shuffled' is not defined

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(dataset1_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(dataset1_bow)
print(messages_tfidf.shape)

# Convert spam and ham labels to 0 and 1 (or, vice-versa)
FactorResult = pd.factorize(data_shuffled["label"])
data_shuffled["label"] = FactorResult[0]
data_shuffled.head()

(44869, 198902)


,label,text
20454,0,sheriff clarke weighs violence trump rally san...
12290,1,london reuters one chennai six group exbritish...
15430,1,brussels reuters eu diplomats start sketching ...
19635,0,
6370,1,washington reuters donald nominee head cia sou...


In [ ]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, data_shuffled["label"], test_size=0.2
)

print(f"train dataset features size: {msg_train.shape}")
print(f"train dataset label size: {label_train.shape}")

print(f"test dataset features size: {msg_test.shape}")
print(f"test dataset label size: {label_test.shape}")

# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)
clf.save_model("CLF_model.json")

# Make predictions
predict_train = clf.predict(msg_train)

print(
    f"Accuracy of Train dataset: {metrics.accuracy_score(label_train, predict_train):0.3f}"
)

train dataset features size: (35895, 198902)
train dataset label size: (35895,)
test dataset features size: (8974, 198902)
test dataset label size: (8974,)
Accuracy of Train dataset: 1.000


In [ ]:
print(messages_tfidf.shape)
print(dataset1_bow.shape)

(44869, 198902)
(44869, 198902)


In [ ]:
NYTwebsite_text = """On his first presidential visit to Saudi Arabia, Mr. Biden received an unsmiling fist bump from the crown prince before meeting King Salman. Their body language is being parsed amid tensions over the Saudi human rights record. President Biden used his visit to Israel this week to bolster the blossoming relationship between Israel and a handful of Arab states. But it is Mr. Biden’s own relationship with Saudi Arabia that looms largest over the second part of his four-day trip, and the kingdom’s welcome of him was being closely watched as an indication of how the overall visit might unfold.

As Air Force One landed, just before 6 p.m. local time, the Saudi reception, while led by a senior member of the royal family, was decidedly lacking in pomp. Lining the lilac carpet were a small number of uniformed security officers bearing swords, but even fewer than were dispatched to welcome President Barack Obama when he arrived in the kingdom to a chilly reception in 2016.

Mr. Biden emerged from the plane and made his way down the stairs for his first visit to Saudi Arabia as president, wearing his aviator shades, waving and looking serious. He was greeted by Princess Reema bint Bandar Al Saud, the Saudi ambassador to Washington, and Prince Khalid Al Faisal, a senior member of the royal family who is the governor of Mecca and is close to King Salman.

The president climbed into his limousine just two minutes after disembarking. He was  taken directly to meet the king.

Arriving at the al-Salam palace, Mr. Biden was met by Crown Prince Mohammed bin Salman, and the two exchanged an apparently wordless fist bump before the prince led the president inside.

The Saudi-owned Al Arabiya satellite channel broadcast images of Mr. Biden being warmly welcomed by King Salman, and shaking his hand. Other images showed Mr. Biden and the king sitting next to each other in armchairs in an ornate room during a meeting, with other U.S. and Saudi officials, including Prince Mohammed, sitting nearby.

Later, Mr. Biden is to hold a “working session” along with his team with Prince Mohammed and various Saudi ministers.Biden’s arrival in Saudi Arabia was polite but perfunctory compared with the enthusiastic greeting President Donald J. Trump received in 2017 when King Salman welcomed him on the tarmac. Mr. Trump made Saudi Arabia his first overseas destination as president, a sharp break with tradition and a sign of how closely he would align his administration with the Saudis.

Even though Mr. Biden vowed as a candidate to punish Saudi Arabia over Mr. Khashoggi’s murder by making the kingdom a “pariah,” the president decided that it was worth the political cost of traveling there this week to counter Chinese influence, press for additional oil production and encourage closer ties with Israel.

In other cases, recently including Cuba and Venezuela, Mr. Biden has stressed that his administration is making democracy and respect for human rights the paramount consideration for dealing with other nations’ leaders. But on Thursday in Jerusalem, he said he was going to Saudi Arabia in order to promote U.S. interests.

Those include getting the kingdom to pump more oil from its somewhat modest spare capacity."""
dict = {'label':['unknown'],
        'text':[NYTwebsite_text],
       }
WP = pd.DataFrame(dict)
WP["text"] = WP["text"].apply(text_preprocess)

# Convert messages (as lists of string tokens) to strings
WP["text"] = WP["text"].agg(lambda x: " ".join(map(str, x)))
print(WP.head())

# Initialize count vectorizer
vectiz = CountVectorizer()
bow_trans = vectiz.fit(WP["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectiz.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectiz.vocabulary_)}")

# Convert strings to vectors using BoW
WP_bow = bow_trans.transform(WP["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {WP_bow.shape}")
print(f"Amount of non-zero occurrences: {WP_bow.nnz}")

tfidf_trans = TfidfTransformer().fit(WP_bow)

# Transform entire BoW into tf-idf corpus
WP_tfidf = tfidf_trans.transform(WP_bow)
#print(clf.predict(WP_tfidf))

#prediction = clf.predict(WP_tfidf)
#print(prediction)
print(msg_train)


     label                                               text
0  unknown  first presidential visit saudi arabia mr biden...
20 BOW Features: ['bearing', 'biden', 'bin', 'bint', 'blossoming', 'body', 'bolster', 'break', 'broadcast', 'bump', 'candidate', 'capacity', 'carpet', 'cases', 'channel', 'chilly', 'chinese', 'climbed', 'close', 'closely']
Total number of vocab words: 214
Shape of sparse matrix: (1, 214)
Amount of non-zero occurrences: 214
  (0, 2722)	0.06959839237859877
  (0, 3609)	0.17982199714902203
  (0, 4130)	0.08077802753143525
  (0, 10848)	0.04988820325808513
  (0, 11668)	0.14767776648691322
  (0, 15068)	0.05321019110868219
  (0, 16735)	0.06015266876749907
  (0, 20286)	0.08782879712381554
  (0, 22551)	0.09585195343529387
  (0, 22577)	0.0645642653929726
  (0, 25183)	0.053861511505976956
  (0, 25282)	0.1279474895158867
  (0, 26394)	0.12434873495168294
  (0, 26670)	0.07663824923599856
  (0, 26976)	0.054587562030027854
  (0, 29099)	0.11320232571633453
  (0, 31979)	0.19214966900

In [ ]:
print(data_shuffled["text"].head(10))

# print the overall accuracy of the model
label_predictions = clf.predict(msg_test)
print(f"Accuracy of the model: {metrics.accuracy_score(label_test, label_predictions):0.3f}")

# an example prediction
print(msg_test.shape)
print(clf.predict(tfidf_transformer.transform(bow_transformer.transform([data_shuffled["text"][9]]))))

print(
    "predicted:",
    clf.predict(
        tfidf_transformer.transform(bow_transformer.transform([data_shuffled["text"][9]]))
    )[0],
)

print("expected:", data_shuffled["label"][9])



20454    sheriff clarke weighs violence trump rally san...
12290    london reuters one chennai six group exbritish...
15430    brussels reuters eu diplomats start sketching ...
19635                                                     
6370     washington reuters donald nominee head cia sou...
1461     washington reuters us health human services se...
821      washingtonkhobar saudi arabia reuters us presi...
10819    white house confirmed fox business priebus wou...
16254    kinshasa reuters democratic republic congo mus...
528      washington reuters republican senator rand pau...
Name: text, dtype: object
Accuracy of the model: 0.998
(1, 198902)


AttributeError: 'Series' object has no attribute 'lower'